## Trading Algoritmico con Python

### Extraccion de datos

In [42]:
# importacion de librerias
import pandas as pd
import hvplot.pandas
import numpy as np
import pandas_ta as ta
import yfinance as yf

In [35]:
# Eleccion de la compañia
aapl = yf.Ticker('NVDA') # Empresas APPLE Inc.

# Periodo de tiempo que se desea analizar
aapl_historical = aapl.history(start='2022-01-2', end='2024-08-2', interval='1d')

# Creacion de nuevo dataframe con datos 'Close y Date'
signals_df = aapl_historical.copy()
signals_df = signals_df.loc[:,['Close']]
signals_df


,Close
Date,
2022-01-03 00:00:00-05:00,30.075632
2022-01-04 00:00:00-05:00,29.245884
2022-01-05 00:00:00-05:00,27.562428
2022-01-06 00:00:00-05:00,28.135561
2022-01-07 00:00:00-05:00,27.205963
...,...
2024-07-26 00:00:00-04:00,113.059998
2024-07-29 00:00:00-04:00,111.589996
2024-07-30 00:00:00-04:00,103.730003


### Establecimiento de señales con medias moviles simples

In [38]:
# Marcos de tiempo
short_window = 50
long_window = 100

# Generacion de las medias moviles
signals_df['SMA_50'] = ta.sma(signals_df['Close'], short_window)
signals_df['SMA_100'] = ta.sma(signals_df['Close'], long_window)
signals_df['Signal'] = 0.0

# Generacion de señales de trading (0 cuando el SMA50 esta debajo de SMA100 y 1 cuando el SMA50 cruza o esta por encima del SMA100)
signals_df['Signal'][short_window:] = np.where(signals_df['SMA_50'][short_window:] > signals_df['SMA_100'][short_window:], 1.0, 0.0)

# Puntos en el tiempo en los que se debe tomar una posicion 1 o -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Dataframe
signals_df

,Close,SMA_50,SMA_100,Signal,Entry/Exit
Date,,,,,
2022-01-03 00:00:00-05:00,30.075632,NaN,NaN,0.0,NaN
2022-01-04 00:00:00-05:00,29.245884,NaN,NaN,0.0,0.0
2022-01-05 00:00:00-05:00,27.562428,NaN,NaN,0.0,0.0
2022-01-06 00:00:00-05:00,28.135561,NaN,NaN,0.0,0.0
2022-01-07 00:00:00-05:00,27.205963,NaN,NaN,0.0,0.0
...,...,...,...,...,...
2024-07-26 00:00:00-04:00,113.059998,118.849688,103.400772,1.0,0.0
2024-07-29 00:00:00-04:00,111.589996,119.189043,103.657102,1.0,0.0
2024-07-30 00:00:00-04:00,103.730003,119.376618,103.807475,1.0,0.0


### Generacion de las señales de trading

In [47]:
# Filtrar datos para puntos de entrada y salida
exit_points = signals_df[signals_df['Entry/Exit'] == -1.0]
entry_points = signals_df[signals_df['Entry/Exit'] == 1.0]

# índice adecuado para la visualización
signals_df.reset_index(inplace=True)  # Esto hará que el índice sea una columna regular llamada 'index'

# Visualización de la posición de salida en relación con el precio de cierre
exit_plot = exit_points.hvplot.scatter(
    x='index',  # Ahora 'index' es una columna del DataFrame
    y='Close',
    color='red',
    label='Exit',
    legend=True,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualización de la posición de entrada en relación con el precio de cierre
entry_plot = entry_points.hvplot.scatter(
    x='index',  # Ahora 'index' es una columna del DataFrame
    y='Close',
    color='green',
    label='Entry',
    legend=True,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualización del precio de cierre de la inversión
security_close_plot = signals_df[['index', 'Close']].hvplot(
    x='index',  # Ahora 'index' es una columna del DataFrame
    y='Close',
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualización de promedios móviles
moving_avgs_plot = signals_df[['index', 'SMA_50', 'SMA_100']].hvplot(
    x='index',  # Ahora 'index' es una columna del DataFrame
    ylabel='Price in $',
    width=1000,
    height=400
)

# Superponer gráficos
entry_exit_plot = security_close_plot * moving_avgs_plot * entry_plot * exit_plot

# Mostrar gráfico
entry_exit_plot

:Overlay
   .Curve.I       :Curve   [index]   (Close)
   .NdOverlay.I   :NdOverlay   [Variable]
      :Curve   [index]   (value)
   .Scatter.Entry :Scatter   [index]   (Close)
   .Scatter.Exit  :Scatter   [index]   (Close)